<a href="https://colab.research.google.com/github/gorech1/hyperT/blob/main/fundus_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#import libraries
import cv2
import numpy as np
import logging
import os
import logging.config

from os import listdir
from os.path import isfile, join

In [ ]:
#los datos(circulos)los tengo en el drive asociado a la cuenta gcp.gorech1
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
#"Google Cloud Storage" data access  
project_id = 'pr1eye'
!gcloud config set project {project_id}

#Los archivos de GCS se almacenan en segmentos. You can make a bucket using the make bucket (mb) command. GCP buckets must have a universally unique name, so use the preinstalled uuid library to generate a Universally Unique ID:.
import uuid
bucket_name = f'fundus_diseases_bucket-{uuid.uuid1()}'
!gsutil mb gs://{bucket_name}


#Para poder acceder a GCS, debemos autenticarnos.
from google.colab import auth
auth.authenticate_user()




Updated property [core/project].
Creating gs://fundus_diseases_bucket-b95e2e24-e31e-11ec-b67f-0242ac1c0002/...


In [31]:
!gsutil cp gs://fundus_diseases_bucket/hyperT/* gs://{bucket_name}/hyperT/

Copying gs://fundus_diseases_bucket/hyperT/1045_left.jpg [Content-Type=image/jpeg]...
Copying gs://fundus_diseases_bucket/hyperT/1045_right.jpg [Content-Type=image/jpeg]...
Copying gs://fundus_diseases_bucket/hyperT/1046_left.jpg [Content-Type=image/jpeg]...
Copying gs://fundus_diseases_bucket/hyperT/1046_right.jpg [Content-Type=image/jpeg]...
\ [4 files][500.4 KiB/500.4 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fundus_diseases_bucket/hyperT/1048_left.jpg [Content-Type=image/jpeg]...
Copying gs://fundus_diseases_bucket/hyperT/1048_right.jpg [Content-Type=image/jpeg]...
Copying gs://fundus_diseases_bucket/hyperT/1081_left.jpg [Content-Type=image/jpeg]...
Copying gs://fundus_diseases_bucket/hyperT/1081_right.jpg [Con

In [11]:
#treat image and remove the excess black area of the images 
class ImageCrop:
    def __init__(self, source_folder, destination_folder, file_name):
        self.logger = logging.getLogger('odir')
        self.source_folder = source_folder
        self.destination_folder = destination_folder
        self.file_name = file_name

    def remove_black_pixels(self):
        file = os.path.join(self.source_folder, self.file_name)
        image = cv2.imread(file)

        # Mask of coloured pixels.
        mask = image > 0

        # Coordinates of coloured pixels.
        coordinates = np.argwhere(mask)

        # Binding box of non-black pixels.
        x0, y0, s0 = coordinates.min(axis=0)
        x1, y1, s1 = coordinates.max(axis=0) + 1  # slices are exclusive at the top

        # Get the contents of the bounding box.
        cropped = image[x0:x1, y0:y1]
        # overwrite the same file
        file_cropped = os.path.join(self.destination_folder, self.file_name)
        cv2.imwrite(file_cropped, cropped)

In [44]:
#loop for treat all the  images 
# Note that this will alter the current training image set folder

def process_all_images():
    files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    for file in files:
        logger.debug('Processing image: ' + file)
        ImageCrop(source_folder, destination_folder, file).remove_black_pixels()


if __name__ == '__main__':
    source_folder =f'/content/drive/My Drive/datos/hyperT/'
    #source_folder = f'gs://{bucket_name}/hyperT'
    destination_folder = f'/content/drive/My Drive/datos/hyperT/cropped'
    # create logger
    #logging.config.fileConfig('logging.conf')
    #logger = logging.getLogger('odir')
    process_all_images()

FileNotFoundError: ignored